# 2. Export loaded Detail Data

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [3]:
folder = 'export/detail/'

df = pd.read_csv('export/0. adressen.csv')

In [32]:
re_telefon = text=re.compile("Telefon (.*)")
re_fax = text=re.compile("Fax (.*)")

for i, row in df.iterrows():
    
    try:
        path = 'dump.html'
        f = open(folder + str(row['id']) + '.html', "r")
        data = f.read()
        soup = BeautifulSoup(data, 'html.parser')

        table = soup.find(class_ = 'mitglied')

        #Ansprechperson
        td_ansprechperson = table.find('td', text='Ansprechperson')
        df.loc[i, 'ansprechperson'] = td_ansprechperson.parent.find_all('td')[1].text

        #Email
        td_email = table.find('td', text='Email')
        if td_email:
            df.loc[i, 'email'] = td_email.parent.find_all('td')[1].text

        address = soup.find('address')

        #Telefon
        dom_telefon = address.find("li", text=re_telefon)
        if dom_telefon:
            s_telefon = re.search(re_telefon, dom_telefon.text)
            df.loc[i, 'telefon'] = s_telefon.group(1)

        #Telefax
        dom_fax = address.find("li", text=re_fax)
        if dom_fax:
            s_fax = re.search(re_fax, dom_fax.text)
            df.loc[i, 'telefax'] = s_fax.group(1)

        f.close()
    except:
        print(row['id'])
        raise

In [35]:
df.to_csv('export/1. addressen final.csv', index=False)